In [1]:
from ultralytics import YOLO

In [2]:
model = YOLO("yolov9c.pt")

In [6]:
results = model.predict('./images/input7.png')


image 1/1 c:\Bryan\TA\Integrated\images\input7.png: 384x640 9 cars, 801.2ms
Speed: 4.0ms preprocess, 801.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


In [43]:
float(results[0].boxes.conf[0])

0.9218660593032837

In [7]:
from ultralytics import YOLO
import cv2
import json
import torch

# Load YOLOv9
# Fail, cuda not yet implemented in yolov9
# if torch.cuda.is_available():
#     torch.cuda.set_device(0)
#     model = YOLO("yolov9c.pt").to('cuda')
#     print('Detecting using GPU...')
# else:
#     model = YOLO("yolov9c.pt").to('cpu')
#     print('Detecting using CPU...')

model = YOLO("yolov9c.pt").to('cpu')
print('Detecting using CPU...')

# Read video file
video_name = 'video2'
input_file_name = f'video/{video_name}.mp4'
cap = cv2.VideoCapture(input_file_name)

# get height, width and frame count of the video
width, height = (
        int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    )
no_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
proc_frames = 0

# Define the codec and create VideoWriter object
# fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
# out = cv2.VideoWriter()
# output_file_name = "output_multi.mp4"
# out.open("output_{}.mp4".format(group_number), fourcc, fps, (width, height), True)

frames = []
try:
    for d in range(no_of_frames):
        ret, frame = cap.read()
        if not ret:
            break
        im = frame
        frames.append(im)
except:
    pass
    # Release resources
    cap.release()
    # out.release()

# Detect all frames using YOLOv9
frames_detections = []
results = model.predict(frames)
for result in results:
    frame_detections = []
    for d, box in enumerate(result.boxes):
        accepted_class_names = ['car', 'bus', 'truck']
        class_index = int(result.boxes.cls[d])
        conf = float(result.boxes.conf[d])
        if result.names[class_index] not in accepted_class_names:
            continue
        frame_detections.append(result.boxes.xywh[d].tolist() +  [class_index, conf])
    frames_detections.append(frame_detections)

# Add to frames detections
with open(f'output/{video_name}_yolov9.json', 'w', encoding='utf-8') as f:
    json.dump({ "final_frames_detections": frames_detections }, f, ensure_ascii=False, indent=4)

# frames_detections.append(frame_jump_unit * group_number + proc_frames)

# Loop through list (if empty this will be skipped) and overlay green bboxes
# for i in bboxes:
#     cv2.rectangle(im, (i[0], i[1]), (i[2], i[3]), (0, 255, 0), 3)

# write the frame
# out.write(im)

# Release resources
cap.release()
# out.release()

# with open(f'output/dist_yolo_{group_number}.json', 'w', encoding='utf-8') as f:
#     json.dump({ "frames_detections": frames_detections }, f, ensure_ascii=False, indent=4)
# out.open("output_{}.mp4".format(group_number), fourcc, fps, (width, height), True)

Detecting using CPU...



RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 15571353600 bytes.

In [6]:
import cv2
import json

yolov9_file_name = f'output/{video_name}_yolov9.mp4'

# Draw bounding box to video
cap = cv2.VideoCapture(input_file_name)
with open(f'output/{video_name}_yolov9.json') as f:
    detections = json.load(f)['final_frames_detections']

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width, height = (
            int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
)
out = cv2.VideoWriter()
out.open(yolov9_file_name, fourcc, fps, (width, height), True)

proc_frames = 0
try:
    while proc_frames < len(detections):
        ret, frame = cap.read()
        if not ret:
            break

        im = frame
        # Loop through list (if empty this will be skipped) and overlay green bboxes
        for d in detections[proc_frames]:
            cv2.rectangle(im, (int(d[0] - d[2]/2), int(d[1] - d[3]/2)), (int(d[0] + d[2]/2), int(d[1] + d[3]/2)), (0, 255, 0), 3)
            cv2.putText(im, f'{d[5]:.2f}', (int(d[0]), int(d[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # write the frame
        out.write(im)

        proc_frames += 1
except Exception as error:
    # Release resources
    print(error)
    cap.release()
    out.release()

out.release()
cap.release()